In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer

pd.set_option('mode.chained_assignment',  None)

In [22]:
r_food_d = pd.read_csv('output/CN_Final_Aug_df.csv', index_col=0)
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat
0,고객,是松柏汤饭吧？我打算预订十人左右。我什么时候应该提前打电话通知您？,"SongbaekdwaejiGukbap, 열 명, 예약, 언제, 전화","松柏汤饭|상호, 十人|인원",예약_문의
0,고객,是Niwa面包店吧？大约在3月25日有一个会议，我想在里面预订，我应该什么时候做呢？,"NiwaBakery, 삼월 이십오일, 모임, 안쪽, 예약, 언제","Niwa面包店|상호, 3月25日|예약일, 里面|위치",예약_문의
0,고객,是千禾猪排吗？我们需要为五个人预订吗？,"CheonhaDonkkaseu, 다섯 명, 예약","千禾猪排|상호, 五个人|인원",예약_문의
0,고객,是Eunjiyakguk吧？如果大约十个人要举行公司晚宴，我们需要提前联系他们吗？,"Eunjiyakguk, 열 명, 회식, 미리, 연락","Eunjiyakguk|상호, 公司晚宴|행사, 十个人|인원",예약_문의
0,고객,是哈诺伊麦居巴默哥里乌沃尔济奥姆，对吗？我们能在同一天吗？,"Hanoimaekjubamgeoriguwoljeom, 당일 낮","哈诺伊麦居巴默哥里乌沃尔济奥姆|상호, 在同一天|시간",예약_문의


In [23]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [24]:
r_food_d['TOKENIZED_SENTENCE'] = r_food_d['SENTENCE'].apply(lambda x: tokenizer.tokenize(x))
r_food_d['ENCODED_SENTENCE'] = r_food_d['SENTENCE'].apply(lambda x: tokenizer.encode(x))
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat,TOKENIZED_SENTENCE,ENCODED_SENTENCE
0,고객,是松柏汤饭吧？我打算预订十人左右。我什么时候应该提前打电话通知您？,"SongbaekdwaejiGukbap, 열 명, 예약, 언제, 전화","松柏汤饭|상호, 十人|인원",예약_문의,"[是, 松, 柏, 汤, 饭, 吧, ？, 我, 打, 算, 预, 订, 十, 人, 左, ...","[101, 3221, 3351, 3377, 3739, 7649, 1416, 8043..."
0,고객,是Niwa面包店吧？大约在3月25日有一个会议，我想在里面预订，我应该什么时候做呢？,"NiwaBakery, 삼월 이십오일, 모임, 안쪽, 예약, 언제","Niwa面包店|상호, 3月25日|예약일, 里面|위치",예약_문의,"[是, [UNK], 面, 包, 店, 吧, ？, 大, 约, 在, 3, 月, 25, 日...","[101, 3221, 100, 7481, 1259, 2421, 1416, 8043,..."
0,고객,是千禾猪排吗？我们需要为五个人预订吗？,"CheonhaDonkkaseu, 다섯 명, 예약","千禾猪排|상호, 五个人|인원",예약_문의,"[是, 千, 禾, 猪, 排, 吗, ？, 我, 们, 需, 要, 为, 五, 个, 人, ...","[101, 3221, 1283, 4897, 4343, 2961, 1408, 8043..."
0,고객,是Eunjiyakguk吧？如果大约十个人要举行公司晚宴，我们需要提前联系他们吗？,"Eunjiyakguk, 열 명, 회식, 미리, 연락","Eunjiyakguk|상호, 公司晚宴|행사, 十个人|인원",예약_문의,"[是, [UNK], 吧, ？, 如, 果, 大, 约, 十, 个, 人, 要, 举, 行,...","[101, 3221, 100, 1416, 8043, 1963, 3362, 1920,..."
0,고객,是哈诺伊麦居巴默哥里乌沃尔济奥姆，对吗？我们能在同一天吗？,"Hanoimaekjubamgeoriguwoljeom, 당일 낮","哈诺伊麦居巴默哥里乌沃尔济奥姆|상호, 在同一天|시간",예약_문의,"[是, 哈, 诺, 伊, 麦, 居, 巴, 默, 哥, 里, 乌, 沃, 尔, 济, 奥, ...","[101, 3221, 1506, 6437, 823, 7931, 2233, 2349,..."


In [25]:
entity_slot = []
slot_vocab = set()

for vals in r_food_d.values:

    # raw_entity = r_food_d.iloc[i]['지식베이스']
    raw_entity = vals[3]
    encoded_sentence = vals[-1][1:-1]

    if type(raw_entity)==float: # 없는 경우
    
        entity_slot.append(['O']* len(encoded_sentence))
    
        continue

    split_entity = raw_entity.split(', ')

    raw_entity_label = []
    raw_entity_name = []

    if len(split_entity)>1: # 여러개 ENTITY
 
        for k in split_entity:      

            entity_name = k.split('|')[0]
            entity_label = k.split('|')[1]

            raw_entity_name.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(entity_name)))
            raw_entity_label.append(entity_label)
   
    else:

        split_entity = raw_entity.split('|')

        for idx, NAME_or_LABEL in enumerate(split_entity):
            
            # [이름, 라벨]
            if idx % 2 == 0: # ENTITY_NAME

                entity_name = NAME_or_LABEL
                raw_entity_name.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(entity_name)))

            else: # ENTITY_LABEL

                entity_label = NAME_or_LABEL
                raw_entity_label.append(NAME_or_LABEL) 

    # sentence_token = ', '.join([str(ENCODED_SENT) for ENCODED_SENT in encoded_sentence])
   
    entity_replace_loc = []
    entity_replace_label = []
    
    for n,l in zip(raw_entity_name, raw_entity_label):

        token_num_list = [str(token_num) for token_num in n]
        bi_list = ['B-' + l if idx == 0 else 'I-'+ l for idx in range(len(token_num_list))]
        slot_vocab.add('B-' + l)
        slot_vocab.add('I-' + l)

        joined_t = ', '.join(token_num_list)
        joined_bi_list = ', '.join(bi_list)

        entity_replace_loc.append(joined_t)
        entity_replace_label.append(joined_bi_list)

    loc_label_dic = {k:v for k,v in zip(entity_replace_loc, entity_replace_label)}

    entity = ['O']* len(encoded_sentence)

    for idx, t in enumerate(entity_replace_loc):

        a = 0
        b = 0

        while a < len(encoded_sentence):

            tmp_loc = t.split(', ')
            tmp_loc = [int(t) for t in tmp_loc]

            if encoded_sentence[a] == tmp_loc[0]:

                if encoded_sentence[a:a+len(tmp_loc)] == tmp_loc:
                    
                    entity[a:a+len(tmp_loc)] = loc_label_dic[str(tmp_loc).replace('[','').replace(']','')].split(', ')

                    a += len(tmp_loc)
                    
                else:

                    a += 1

            else:

                a += 1

            
    entity_slot.append(entity)
    
slot_vocab = sorted(list(slot_vocab), key=lambda x: (x[2:], x[:2]))

r_food_d['ENTITY_SLOT'] = entity_slot

In [26]:
r_food_d['length'] = r_food_d['TOKENIZED_SENTENCE'].apply(lambda x: len(x))
r_food_d['length'].max()

67

In [27]:
r_food_d['ES_LEN'] = r_food_d['ENTITY_SLOT'].apply(lambda x:len(x))

In [28]:
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat,TOKENIZED_SENTENCE,ENCODED_SENTENCE,ENTITY_SLOT,length,ES_LEN
0,고객,是松柏汤饭吧？我打算预订十人左右。我什么时候应该提前打电话通知您？,"SongbaekdwaejiGukbap, 열 명, 예약, 언제, 전화","松柏汤饭|상호, 十人|인원",예약_문의,"[是, 松, 柏, 汤, 饭, 吧, ？, 我, 打, 算, 预, 订, 十, 人, 左, ...","[101, 3221, 3351, 3377, 3739, 7649, 1416, 8043...","[O, B-상호, I-상호, I-상호, I-상호, O, O, O, O, O, O, ...",33,33
0,고객,是Niwa面包店吧？大约在3月25日有一个会议，我想在里面预订，我应该什么时候做呢？,"NiwaBakery, 삼월 이십오일, 모임, 안쪽, 예약, 언제","Niwa面包店|상호, 3月25日|예약일, 里面|위치",예약_문의,"[是, [UNK], 面, 包, 店, 吧, ？, 大, 约, 在, 3, 月, 25, 日...","[101, 3221, 100, 7481, 1259, 2421, 1416, 8043,...","[O, B-상호, I-상호, I-상호, I-상호, O, O, O, O, O, B-예...",38,38
0,고객,是千禾猪排吗？我们需要为五个人预订吗？,"CheonhaDonkkaseu, 다섯 명, 예약","千禾猪排|상호, 五个人|인원",예약_문의,"[是, 千, 禾, 猪, 排, 吗, ？, 我, 们, 需, 要, 为, 五, 个, 人, ...","[101, 3221, 1283, 4897, 4343, 2961, 1408, 8043...","[O, B-상호, I-상호, I-상호, I-상호, O, O, O, O, O, O, ...",19,19
0,고객,是Eunjiyakguk吧？如果大约十个人要举行公司晚宴，我们需要提前联系他们吗？,"Eunjiyakguk, 열 명, 회식, 미리, 연락","Eunjiyakguk|상호, 公司晚宴|행사, 十个人|인원",예약_문의,"[是, [UNK], 吧, ？, 如, 果, 大, 约, 十, 个, 人, 要, 举, 行,...","[101, 3221, 100, 1416, 8043, 1963, 3362, 1920,...","[O, B-상호, O, O, O, O, O, O, B-인원, I-인원, I-인원, ...",31,31
0,고객,是哈诺伊麦居巴默哥里乌沃尔济奥姆，对吗？我们能在同一天吗？,"Hanoimaekjubamgeoriguwoljeom, 당일 낮","哈诺伊麦居巴默哥里乌沃尔济奥姆|상호, 在同一天|시간",예약_문의,"[是, 哈, 诺, 伊, 麦, 居, 巴, 默, 哥, 里, 乌, 沃, 尔, 济, 奥, ...","[101, 3221, 1506, 6437, 823, 7931, 2233, 2349,...","[O, B-상호, I-상호, I-상호, I-상호, I-상호, I-상호, I-상호, ...",29,29


In [29]:
path = 'split_dataset' # 67

In [30]:
file_name = './{}/slot_label.txt'.format(path)
with open(file_name, 'w+') as f_w:
    additional_tokens = ["PAD", "UNK", "O"]
    for token in additional_tokens:
            f_w.write(token + '\n')

    for slot in slot_vocab:
        f_w.write(slot + '\n')

intent_cat_list = r_food_d['intent_cat'].unique().tolist()
intent_cat_list = sorted(intent_cat_list)


file_name = './{}/intent_label.txt'.format(path)

with open(file_name, 'w+') as f_w:
    additional_tokens = ["UNK"]
    for token in additional_tokens:
            f_w.write(token + '\n')

    for intent in intent_cat_list:
        f_w.write(intent + '\n')

In [31]:
total_dataset = r_food_d[['SENTENCE','intent_cat','TOKENIZED_SENTENCE','ENCODED_SENTENCE','ENTITY_SLOT']]

train_dataset, test_dataset = train_test_split(total_dataset, train_size= 0.8, random_state=42, stratify=total_dataset['intent_cat'])

train_dataset, dev_dataset = train_test_split(train_dataset, train_size= 0.75, random_state=42, stratify=train_dataset['intent_cat'])

In [32]:
train_seq_in = []

for seq_in in train_dataset['SENTENCE']:

    train_seq_in.append(seq_in)

file_name = './{}/train/seq.in'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(train_seq_in)) 

test_seq_in = []

for seq_in in test_dataset['SENTENCE']:

    test_seq_in.append(seq_in)

file_name = './{}/test/seq.in'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(test_seq_in)) 

dev_seq_in = []

for seq_in in dev_dataset['SENTENCE']:

    dev_seq_in.append(seq_in)

file_name = './{}/dev/seq.in'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(dev_seq_in)) 

train_label = []

for ic in train_dataset['intent_cat']:

    train_label.append(ic)

file_name = './{}/train/label'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(train_label)) 

test_label = []

for ic in test_dataset['intent_cat']:

    test_label.append(ic)

file_name = './{}/test/label'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(test_label)) 

dev_label = []

for ic in dev_dataset['intent_cat']:

    dev_label.append(ic)

file_name = './{}/dev/label'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(dev_label)) 

train_seq_out = []

for seq_out_list in train_dataset['ENTITY_SLOT']:

    temp_seq_out = []
    
    for out in seq_out_list:

        temp_seq_out.append(out)

    temp_seq_out = ' '.join(temp_seq_out)

    train_seq_out.append(temp_seq_out)

file_name = './{}/train/seq.out'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(train_seq_out)) 

test_seq_out = []

for seq_out_list in test_dataset['ENTITY_SLOT']:

    temp_seq_out = []

    for out in seq_out_list:

        temp_seq_out.append(out)

    temp_seq_out = ' '.join(temp_seq_out)

    test_seq_out.append(temp_seq_out)
    

file_name = './{}/test/seq.out'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(test_seq_out)) 

dev_seq_out = []

for seq_out_list in dev_dataset['ENTITY_SLOT']:

    temp_seq_out = []
    
    for out in seq_out_list:

        temp_seq_out.append(out)

    temp_seq_out = ' '.join(temp_seq_out)

    dev_seq_out.append(temp_seq_out)

file_name = './{}/dev/seq.out'.format(path)

with open(file_name, 'w+') as file:
    file.write('\n'.join(dev_seq_out))